# RNASeq pipeline analysis

We will perform differential expression (DE) analysis of the RNA-Seq sequencing of Escherichia Coli K-12 in stressful condition. 
https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-6993/

We compare here 2 biological conditions:
- WT - Bacteria grown in liquid cultures
- MazF - Bacteria grown in liquid cultures with 2 hours of MazF induction

Biological protocol:
After RNA extraction, ribosomal RNAs have been removed using Ribo-Zero rRNA Removal Kit. This step is critical as rRNA account for 90% to 95% of total RNA. Remaining RNA are sequenced using Illumina MiSeq. Paired-end sequecing is used so each dataset contain two fastq files. 

## Dataset download

Download all sequencing files. For each biological condition there are 2 biological replicates, and for each replicate 2 fast files (paired-end sequencing).
You should end up with 8 files in your folder `DataEcoli`:
- WT_1_R1.fastq.gz
- WT_1_R2.fastq.gz
- WT_2_R1.fastq.gz
- WT_2_R2.fastq.gz
- MazF_1_R1.fastq.gz
- MazF_1_R2.fastq.gz
- MazF_2_R1.fastq.gz
- MazF_2_R2.fastq.gz


In [8]:
%%bash
mkdir "DataEcoli"
# Download WT dataset (replicate 1)
wget -O "DataEcoli/WT_1_R1.fastq.gz" ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR268/005/ERR2686025/ERR2686025_1.fastq.gz
wget -O "DataEcoli/WT_1_R2.fastq.gz" ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR268/005/ERR2686025/ERR2686025_2.fastq.gz
# Download WT dataset (replicate 2)
wget -O "DataEcoli/WT_2_R1.fastq.gz" ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR268/006/ERR2686026/ERR2686026_1.fastq.gz
wget -O "DataEcoli/WT_2_R2.fastq.gz" ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR268/006/ERR2686026/ERR2686026_2.fastq.gz
    
# Download MazF dataset (replicate 1)
wget -O "DataEcoli/MazF_1_R1.fastq.gz" ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR268/007/ERR2686027/ERR2686027_1.fastq.gz
wget -O "DataEcoli/MazF_1_R2.fastq.gz" ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR268/007/ERR2686027/ERR2686027_2.fastq.gz
# Download MazF dataset (replicate 2)
wget -O "DataEcoli/MazF_2_R1.fastq.gz" ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR268/008/ERR2686028/ERR2686028_1.fastq.gz
wget -O "DataEcoli/MazF_2_R2.fastq.gz" ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR268/008/ERR2686028/ERR2686028_2.fastq.gz

Process is interrupted.


## Genome download

We have now to download the genome file of Escherichis coli K-12. We download first the sequence (fasta file) and then the annotation (gff file).

In [29]:
%%bash
mkdir Genome
wget -O "Genome/Ecoli_K12.fna.gz" ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
wget -O "Genome/Ecoli_K12.gtf.gz" ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gtf.gz

mkdir: Genome: File exists
--2019-11-19 22:57:08--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
           => « Genome/Ecoli_K12.fna.gz »
Résolution de ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)… 2607:f220:41e:250::13, 130.14.250.10
Connexion à ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|2607:f220:41e:250::13|:21… connecté.
Ouverture de session en tant que anonymous… Session établie.
==> SYST ... terminé.    ==> PWD ... terminé.
==> TYPE I ... terminé.  ==> CWD (1) /genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2 ... terminé.
==> SIZE GCF_000005845.2_ASM584v2_genomic.fna.gz ... 1379902
==> EPSV ... terminé.    ==> RETR GCF_000005845.2_ASM584v2_genomic.fna.gz ... terminé.
Taille : 1379902 (1,3M) (non certifiée)

     0K .......... .......... .......... .......... ..........  3%  147K 9s
    50K .......... .......... .......... .......... ..........  7%  340K 6s
   100K .......... .......... .......... ....

Uncompress genome files, either by using gunzip or just clicking on it in your OS.

In [30]:
%%bash
gunzip -d Genome/Ecoli_K12.fna.gz
gunzip -d Genome/Ecoli_K12.gtf.gz

## Clean reads

Before running mapping of reads we need to clean them by running "trimming" operation

In [33]:
%%bash
mkdir FastQ
read1Adapter=AGATCGGAAGAGCACACGTCTGAACTCCAGTCA
read2Adapter=AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
cutadapt -j 8 -b $read1Adapter -B $read2Adapter -o FastQ/WT_1_R1.fastq -p FastQ/WT_1_R2.fastq DataEcoli/WT_1_R1.fastq.gz DataEcoli/WT_1_R2.fastq.gz
cutadapt -j 8 -b $read1Adapter -B $read2Adapter -o FastQ/WT_2_R1.fastq -p FastQ/WT_2_R2.fastq DataEcoli/WT_2_R1.fastq.gz DataEcoli/WT_2_R2.fastq.gz
cutadapt -j 8 -b $read1Adapter -B $read2Adapter -o FastQ/MazF_1_R1.fastq -p FastQ/MazF_1_R2.fastq DataEcoli/MazF_1_R1.fastq.gz DataEcoli/MazF_1_R2.fastq.gz
cutadapt -j 8 -b $read1Adapter -B $read2Adapter -o FastQ/MazF_2_R1.fastq -p FastQ/MazF_2_R2.fastq DataEcoli/MazF_2_R1.fastq.gz DataEcoli/MazF_2_R2.fastq.gz

mkdir: FastQ: File exists
sh: line 4: cutadapt: command not found
sh: line 5: cutadapt: command not found
sh: line 6: cutadapt: command not found
sh: line 7: cutadapt: command not found


CalledProcessError: Command 'b'mkdir FastQ\nread1Adapter=AGATCGGAAGAGCACACGTCTGAACTCCAGTCA\nread2Adapter=AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT\ncutadapt -b $read1Adapter -B $read2Adapter -o FastQ/WT_1_R1.fastq -p FastQ/WT_1_R2.fastq DataEcoli/WT_1_R1.fastq DataEcoli/WT_1_R2.fastq\ncutadapt -b $read1Adapter -B $read2Adapter -o FastQ/WT_2_R1.fastq -p FastQ/WT_2_R2.fastq DataEcoli/WT_2_R1.fastq DataEcoli/WT_2_R2.fastq\ncutadapt -b $read1Adapter -B $read2Adapter -o FastQ/MazF_1_R1.fastq -p FastQ/MazF_1_R2.fastq DataEcoli/MazF_1_R1.fastq DataEcoli/MazF_1_R2.fastq\ncutadapt -b $read1Adapter -B $read2Adapter -o FastQ/MazF_2_R1.fastq -p FastQ/MazF_2_R2.fastq DataEcoli/MazF_2_R1.fastq DataEcoli/MazF_2_R2.fastq\n'' returned non-zero exit status 127.

Check quality of the reads

In [34]:
%%bash
mkdir FastQC
fastqc -t 8 -o FastQC FastQ/WT_1_R1.fastq FastQ/WT_1_R2.fastq FastQ/WT_2_R1.fastq FastQ/WT_2_R2.fastq
fastqc -t 8 -o FastQC FastQ/MazF_1_R1.fastq FastQ/MazF_1_R2.fastq FastQ/MazF_2_R1.fastq FastQ/MazF_2_R2.fastq

bash: line 2: fastqc: command not found
bash: line 3: fastqc: command not found


CalledProcessError: Command 'b'mkdir FastQC\nfastqc -o FastQC FastQ/WT_1_R1.fastq FastQ/WT_1_R2.fastq FastQ/WT_2_R1.fastq FastQ/WT_2_R2.fastq\nfastqc -o FastQC FastQ/MazF_1_R1.fastq FastQ/MazF_1_R2.fastq FastQ/MazF_2_R1.fastq FastQ/MazF_2_R2.fastq\n'' returned non-zero exit status 127.

## Reads mapping

We will first create the index file for Escherichia Coli K12 genome. This is mandatory for running reads mapping.

In [31]:
%%bash
bowtie2-build --threads 8 Genome/Ecoli_K12.fna Genome/Ecoli_K12

bash: line 1: bowtie2-build: command not found


CalledProcessError: Command 'b'bowtie2-build Genome/Ecoli_K12.fna Genome/Ecoli_K12\n'' returned non-zero exit status 127.

We can now finally map the reads.

In [ ]:
%%bash
mkdir Mapping
bowtie2 -p 8 -x Genome/Ecoli_K12 -1 FastQ/WT_1_R1.fastq -2 FastQ/WT_1_R2.fastq -S Mapping/WT_1.sam > Mapping/WT_1.log
bowtie2 -p 8 -x Genome/Ecoli_K12 -1 FastQ/WT_2_R1.fastq -2 FastQ/WT_2_R2.fastq -S Mapping/WT_2.sam > Mapping/WT_2.log &
bowtie2 -p 2 -x Genome/Ecoli_K12 -1 FastQ/MazF_1_R1.fastq -2 FastQ/MazF_1_R2.fastq -S Mapping/MazF_1.sam > Mapping/MazF_1.log &
bowtie2 -p 2 -x Genome/Ecoli_K12 -1 FastQ/MazF_2_R1.fastq -2 FastQ/MazF_2_R2.fastq -S Mapping/MazF_2.sam > Mapping/MazF_2.log &


You should have now 4 .sam files corresponding to your reads mapped on the E. coli genome. For convenience we will convert this files to .bam file which are lighter version of theses files (indexed and binary).

In [ ]:
%%bash
samFile=Mapping/WT_1
samtools view -b -q 1 $samFile.sam > ${samFile}_raw.bam
samtools sort -o ${samFile}.bam ${samFile}_raw.bam
samtools index ${samFile}.bam
rm ${samFile}_raw.bam

In [ ]:
%%bash
samFile=Mapping/WT_2
samtools view -b -q 1 $samFile.sam > ${samFile}_raw.bam
samtools sort -o ${samFile}.bam ${samFile}_raw.bam
samtools index ${samFile}.bam
rm ${samFile}_raw.bam

In [ ]:
%%bash
samFile=Mapping/MazF_1
samtools view -b -q 1 $samFile.sam > ${samFile}_raw.bam
samtools sort -o ${samFile}.bam ${samFile}_raw.bam
samtools index ${samFile}.bam
rm ${samFile}_raw.bam

In [ ]:
%%bash
samFile=Mapping/MazF_2
samtools view -b -q 1 $samFile.sam > ${samFile}_raw.bam
samtools sort -o ${samFile}.bam ${samFile}_raw.bam
samtools index ${samFile}.bam
rm ${samFile}_raw.bam

## Reads counting

Now that all reads are mapped, we use .bam files to count reads in all genes of E. coli. The annotation file Ecoli_KL12.gtf will be used.

In [21]:
%%bash
mkdir GeneCount
featureCounts -t "gene" -a Genome/Ecoli_K12.gtf -o GeneCount/WT_1.txt Mapping/WT_1.bam
featureCounts -t "gene" -a Genome/Ecoli_K12.gtf -o GeneCount/WT_2.txt Mapping/WT_2.bam
featureCounts -t "gene" -a Genome/Ecoli_K12.gtf -o GeneCount/MazF_1.txt Mapping/MazF_1.bam
featureCounts -t "gene" -a Genome/Ecoli_K12.gtf -o GeneCount/MazF_2.txt Mapping/MazF_2.bam

UsageError: Cell magic `%%bashfeatureCounts` not found.


## Quality control

Before doing the differential expression analysis we will perform a multi QC analysis. We will thus control that everything went well.

In [ ]:
%%bash
mkdir MultiQC
multiqc -f -i EcoliRNASeq -n EcoliRNASeq -o MultiQC .

You check your results in `MultiQC/EcoliRNASeq.html` file

## Differential Expression analysis

We are now reaching the last part of our analysis. We will compare the two MazF replicates to the two WT replicates. We expect to have a list of genes differentially expressed and an html report showing the quality of our analysis.

In [ ]:
%%bash
Rscript RunRNASeqDESeq2.R

A final, `RNASeq_Ecoli_report.html` webpage report has been produced along with tables summarizing the analysis (`MazFvsWT.complete.txt`, `MazFvsWT.up.txt`, `MazFvsWT.down.txt`).

We found:
1359 genes downregulated, and 1296 upregulated genes

# Use cluster computing to speed up computation

(This part only works when you have access to the cluster = not today)

I am creating a bash file containing all information for the cluster (see `WT_1_Mapping.sh`).

Then I run it on the cluster using `qsub` program.

In [5]:
%%bash
qsub -l ncpus=12,mem=40G -N Mapping -e WT_1_Mapping_err.log -o WT_1_Mapping.log WT_1_Mapping.sh

bash: line 1: qsub: command not found


CalledProcessError: Command 'b'qsub -l ncpus=12,mem=40G -N Mapping -e /home/becavin/omicsTD//temp/Mapping/WT_1_Mapping_err.log -o /home/becavin/omicsTD//temp/Mapping/WT_1_Mapping.log /home/becavin/omicsTD//temp/Mapping/WT_1_Mapping.sh\n'' returned non-zero exit status 127.

## Run on all datasets

Now that I have controlled that everything run well, I can use the full power of the cluster and run as much `qsub` as datasets. To do that I have wrapped everything in `RunMapping.sh` which creates `WT_1_Mapping.sh` and the others, and then run the appropriate `qsub`.

The key parameters in `RunMapping.sh`is the number of CPUs and memory which you ask (NB_CPUS=24, MEM_JOB=40G).

In [1]:
%%bash
sh RunMapping.sh WT_1
sh RunMapping.sh WT_2
sh RunMapping.sh MazF_1
sh RunMapping.sh MazF_2

UsageError: %%bash is a cell magic, but the cell body is empty.
